In [1]:
import numpy as np
import nltk
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score
from gensim.utils import simple_preprocess
from gensim.models import TfidfModel, LdaModel
from gensim import corpora
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

categories = ['rec.autos', 'comp.graphics', 'sci.space']
newsgroup = fetch_20newsgroups(subset='all', categories=categories, shuffle=True, remove=('headers', 'footers', 'quotes'))
print(newsgroup.filenames.shape)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhest\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(2950,)


In [2]:
tokenized_documents = [simple_preprocess(text) for text in newsgroup.data]
dictionary = corpora.Dictionary(tokenized_documents)
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents]
modelTf = TfidfModel(bow_corpus)
tf_corpus = [modelTf[corpus_item] for corpus_item in bow_corpus]

In [3]:
def train(X_train, X_test, y_train, y_test, n):
    clf = AdaBoostClassifier(n_estimators=n,algorithm='SAMME')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return f1_score(y_test, y_pred, average='weighted')

In [5]:
for n in [10,15,20,25,30,40,50,60,70,80,90,100,120,140,160,180,200]:
    currArr = []
    mArr = [25,50,75,100,125,150,175,200,250,300,350,400]
    for m in mArr:
        lda = LdaModel(tf_corpus, num_topics=n, id2word=dictionary,passes=15, minimum_probability = 0)
        vectorized_corpus = lda[tf_corpus]
        vectorized_corpus_new = [0 for i in range(2950)]
        for i in range(len(vectorized_corpus)):
            curr = []
            a = vectorized_corpus[i]
            length = len(a)
            curr = [a[j][1] for j in range(length)]
            vectorized_corpus_new[i] = curr
        X_train, X_test, y_train, y_test = train_test_split(vectorized_corpus_new, newsgroup.target, test_size=0.33)
        currArr.append(train(X_train, X_test, y_train, y_test,m))
    currMax = max(currArr)
    print(f"N:{n}, M:{mArr[currArr.index(currMax)]}, result is {currMax}")    

N:10, M:25, result is 0.555690330672063
N:15, M:125, result is 0.5531375254958931
N:20, M:350, result is 0.5738505225604708
N:25, M:200, result is 0.5944404040415348
N:30, M:400, result is 0.5477144931786636
N:40, M:200, result is 0.6258183863647243
N:50, M:50, result is 0.5715473244208591
N:60, M:100, result is 0.6133047601874202
N:70, M:125, result is 0.6204109115463211


KeyboardInterrupt: 